<a href="https://colab.research.google.com/github/AltmannPeter/TopicsReview/blob/main/Topic_K_Appendices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic K - Appendices

## Appendix A: A third party repudiable ZKP-based PoA

In [1]:
%%capture
!pip install ecpy
import secrets
from ecpy.curves    import Curve, Point

An EC setup requires a curve and two generators, $(G, H)$. $G$, is the curve's standard generator, while security requires $H$ to be chosen using a publicly verifiable random process. We need to ensure each Issuer generates a distinct generator and eliminate as many attack vectors as possible.

One illustrative solution is a KEM-seeded Hash-to-Curve ([RFC 9180](https://www.rfc-editor.org/rfc/rfc9180.html) with [RFC 9380](https://www.rfc-editor.org/rfc/rfc9380.html)). The KEM part ensures Issuer distinct generators, while Hash-to-Curve outputs a generator with unknown scalar.

Here, we just pick $H$ at random.

In [2]:
# The curves
crv = Curve.get_curve("secp256r1")
order = crv.order
G  = crv.generator

# 1st blind generator
H_1 = secrets.randbelow(order) * G

# 2nd blind generator
H_2 = secrets.randbelow(order) * G

The PID issuer includes a unique random value, `poa_seed`, in the PID attestation. Note that this value is **NOT** intended to act as a unique identifier for the User, but as a PoA input.

In [3]:
# The PID issuer includes a PoA seed in the attestation
poa_seed = secrets.randbelow(order)

In the next step, two additional distinct Issuers rely on the poa_seed to generate a blinded version each, $H_1, H_2$. Note that only the User sees both blinded values at this step.

In [4]:
# Attestation Issuer 1
blind_poa_input_1 = poa_seed * H_1

#Attestation Issuer 2
blind_poa_input_2 = poa_seed * H_2

The User can now reveal the two blinded PoA values to a Verifier. The Verifier will require the User to generate a PoA between $A$ and $B$.

The User can generate a PoA proof using e.g., a PODLE like the Chaum-Pedersen protocol, which allows the User to prove that it knows two discrete logarithms $y_1 = h_1^r$ and $y_2 = h_2^r$ with respect to two distinct generators $h_1, h_2$.

The Verifier accepts the proof of discrete log equivalence as a proof of association.

In [5]:
# User samples a random nonce
r = secrets.randbelow(order)

# The User shares two commitments with the Verifier
P = r * H_1
Q = r * H_2

# A random challenge is produced either by the Verifier or using a random oracle.
# Here we just sample from random
# A repudiable proof requires Verifier picked c thus IZKP
# A NIZKP with Fiat-Shamir is always third party non-repudiable
c = secrets.randbits(256)

# The user computes s
s = r + c * poa_seed

# The Verifier verifies the PODLE using challenge c, recieved values (s, P, Q), and blinded PoAs
assert (s * H_1 == P + (blind_poa_input_1 * c))
assert (s * H_2 == Q + (blind_poa_input_2 * c))

The ZKP property means that there exists a simulator that can produce proofs indistinguishable from real ones even without knowledge of the secret. This is useful when a Proof of Association (PoA) needs to be repudiable. Works only with IZKP and not NIZKP (the latter does not permit the challenge to be selected before R).

The **real proof**, given $Y = x \cdot H_1$ and $Z = x \cdot H_2$, where the Prover knows $x$ consists of:

* Prover commitments: $P = r \cdot H_1, Q = r \cdot H_2$
* A random challenge $c$
* Prover computation: $s = r + c \cdot x $

The **simulated proof** is:

* Pick $s$ at random
* Compute $P = s \cdot H_1 - c \cdot Y, Q = s \cdot H_2 - c \cdot Z$

The transcript is verified with:

* $s \cdot H_1 = P + c \cdot Y$
* $s \cdot H_2 = Q + c \cdot Z$

Since the simulated transcript is computationally indistinguishable from a real one, a third party cannot determine whether the Prover actually participated. This enables repudiability of the PoA, meaning the Prover can plausibly deny involvement in the proof.

## Appendix B: Simple PoA when WSCD supports arbitrary operations

Alg 1-2 in Verheul presents a way to generate a PoA between two private keys protected by the same WSCD. Verheul's proposal builds on the assumption that the Prover "has full control over both private keys" and can "do arbitrary mathematical operations with these." This is seemingly required because of the limitation to rely on a the curve specified generator $G$ only.

One possible argument raised against Alg 1-2 is that they are overly complex given that the assumption is *full* control. A WSCD that has full control can simplify the PoA.

In [6]:
# Using the same notation as Alg 1-2
p_1 = secrets.randbelow(order)
p_2 = secrets.randbelow(order)

# The two public keys the Prover wants to generate a PoA for
P_1 = p_1 * G
P_2 = p_2 * G

# Instead of an association key, the WSCD outputs a third value z
z = p_1 + p_2

# The verification is now
P_1 + P_2 == z*G

True

The above approach is extendable to more than two private keys. Given the scalar sum $z = \sum_{i=0}^n{p_i}$, the corresponding public key sum is: $\sum_{i=0}^n{P_i} = z \cdot G$, which serves as a combined PoP and PoA.

Since each private key is a random value, their sum is indistinguishable from a randomly chosen scalar from the same group. Afaik, knowledge of $z$ does not leak information about either $p_1$ or $p_2$ without additional information; it is information theoretically secure. If necessary, add a random $r$ to the sum.

An attacker with knowlege of $(P_1, P_2, z)$ cannot extract either $p_1$ or $p_2$ from $(P_1, P_2)$ because that would break ECDLP.

Seemingly, an exhaustive search is the best approach for the attacker in the case of single-use-keys, which is the same as the attacker has for guessing either $p_1$ or $p_2$ from $P_1$ or $P_2$.